### Домашнее задание №7 по курсу «Машинное обучение»: SGD

#### Выполнила Мирейко Наталья

#### Заданиe


Скачайте датасет «Ирисы Фишера» (http://archive.ics.uci.edu/ml/datasets/Iris). В этих данных описаны три вида ирисов. Случайным образом разбейте датасет в соотношении 9 : 1. Первую часть надо будет использовать в качестве обучающей выборки. Вторую — в качестве тестовой (для оценки качества полученной гипотезы). В каждом задании вам необходимо будет построить три модели, по одной модели для каждого вида ирисов. Соответствующая модель должна решать задачу классификации на два класс — принадлежит данный цветок рассматриваемому виду или нет.
1. обучите модель логистической регрессии с регуляризацией Тихонова с помощью SGD-алгоритма. Необходимые параметры подберите с помощью алгоритма k-fold.
2. обучите модель логистической регрессии с регуляризацией Тихонова с помощью алгоритма batch gradient descent. Этот алгоритм выбирает в качестве направления на каждом шаге −∇LS(w(t)). Необходимые параметры подберите с помощью алгоритм k-fold.
3. сравните два полученных решения. Что вы можете сказать о плюсах и минусах каждого из подходов?


#### Решениe


Был скачан dataset (http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) и несколько модифицирован сам файл для удобства работы с ним: значение последней колонки с названиями ирисов (Iris-setosa, Iris-versicolor, Iris-virginica) была заменена на номер группы (1, 2, 3) сооответственно.

То есть, например, строки dataset'а:


5.1,3.5,1.4,0.2,Iris-setosa

7.0,3.2,4.7,1.4,Iris-versicolor

6.3,3.3,6.0,2.5,Iris-virginica


стали выглядеть так:


5.1,3.5,1.4,0.2,1

7.0,3.2,4.7,1.4,2

6.3,3.3,6.0,2.5,3


Ниже представлен код Stochastic Gradient Descent и Batch Gradient Descent алгоритмов.

In [3]:
# Linear Regression With Stochastic Gradient Descent for Wine Quality
from random import seed
from random import randrange
from csv import reader
from math import sqrt
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate root mean squared error
def rmse_metric(actual, predicted):
	sum_error = 0.0
	for i in range(len(actual)):
		prediction_error = predicted[i] - actual[i]
		sum_error += (prediction_error ** 2)
	mean_error = sum_error / float(len(actual))
	return sqrt(mean_error)
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = linear_regression(algorithm, train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		rmse = rmse_metric(actual, predicted)
		scores.append(rmse)
	return scores
 
# Make a prediction with coefficients
def predict(row, coefficients):
	yhat = coefficients[0]
	for i in range(len(row)-1):
		yhat += coefficients[i + 1] * row[i]
	return yhat
 
# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			yhat = predict(row, coef)
			error = yhat - row[-1]
			coef[0] = coef[0] - l_rate * error
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] - l_rate * error * row[i]
			# print(l_rate, n_epoch, error)
	return coef

# Estimate linear regression coefficients using batch gradient descent
def coefficients_bgd(train, l_rate, n_epoch):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		sum_error_row = [0.0 for i in range(len(train[0]))]
		for row in train:			
			yhat = predict(row, coef)
			error = yhat - row[-1]
			coef[0] = coef[0] - l_rate * error
			for i in range(len(row)-1):
				sum_error_row[i] = sum_error_row[i] + error * row[i]	
		for i in range(len(train[0])-1):	
			coef[i + 1] = coef[i + 1] - l_rate * sum_error_row[i]
			# print(l_rate, n_epoch, error)
	return coef

# Linear Regression Algorithm With Passed Gradient Descent Algorithm
def linear_regression(coefficients_algorithm, train, test, l_rate, n_epoch):
	predictions = list()
	coef = coefficients_algorithm(train, l_rate, n_epoch)
	for row in test:
		yhat = predict(row, coef)
		predictions.append(yhat)
	return(predictions)
 
# Linear Regression on iris dataset
seed(1)
# load and prepare data
filename = 'iris.data'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 10
l_rate = 0.01
n_epoch = 50
sgd_scores = evaluate_algorithm(dataset, coefficients_sgd, n_folds, l_rate, n_epoch)
print('SGD - Scores: %s' % sgd_scores)
print('SGD - Mean RMSE: %.3f' % (sum(sgd_scores)/float(len(sgd_scores))))
bgd_scores = evaluate_algorithm(dataset, coefficients_bgd, n_folds, l_rate, n_epoch)
print('BGD - Scores: %s' % bgd_scores)
print('BGD - Mean RMSE: %.3f' % (sum(bgd_scores)/float(len(bgd_scores))))


SGD - Scores: [0.13251110316402398, 0.13179449369581947, 0.12412050494021755, 0.09453538492297843, 0.1070582295656936, 0.10254315828497079, 0.10828878513564613, 0.11583097203104432, 0.13765248765094681, 0.07347102690135655]
SGD - Mean RMSE: 0.113
BGD - Scores: [0.13310466200492324, 0.1445656700672241, 0.10681892092464784, 0.11225471032524002, 0.12906259001943918, 0.06842761913904624, 0.11656402320020395, 0.082459246301777, 0.12313445248139471, 0.10281385971159139]
BGD - Mean RMSE: 0.112


В итоге получили незначительную разницу в значении ошибки RMSE двух алгоритмов (SGD и BGD) - примерно 0.001, это объясняется маленьким значением исходной выборки и тем, что эти алгоритмы - вариации gradient descent алгоритма.

В алгоритме batch gradient descent на каждом шаге мы проходимся по всем примерам ирисов, чтобы посчитать градиент, и обновляем коэффициенты только в конце каждой итерации-шага epoch. Когда исторических примеров не много, все в порядке, но когда их миллионы, для каждого маленького шага к минимуму мы должны проделать миллионы вычислений, и это может занимать долгое время (то есть трудоемко).

Альтернативой этому алгоритму может быть stochastic gradient descent – метод, при котором мы берем какой-то один пример и обновляем значения коэффициентов, ориентируясь только на него. Потом берем следующий пример и обновляем параметры, ориентируясь уже на него. И так далее. Это приводит к тому, что мы не всегда «спускаемся» с холма, иногда мы делаем и шаг вверх или в сторону, но рано или поздно мы достигаем того самого минимума и начинаем кружить вокруг него. Когда значения параметров начинают нас устраивать (достигают нужной нам точности), мы останавливаем спуск. Таким образом, плюсы SGD: не требует хранения всех данных обучения в памяти, а следовательно, хорошо для больших наборов тренировок, позволяет добавлять новые данные в режиме «онлайн».

Так же есть особенности схождения алгоритма: batch gradient descent всегда сходится к минимуму при условии, что используется достаточно маленькое значение l_rate. Stochastic gradient descent в общем виде не сходится к минимуму, но есть его модификации, которые позволяют добиться сходимости.